# MNIST with CNN

## MNIST data set download/import

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np


#Select NVIDIA if available, otherwise cpu:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device = torch.device("cuda:0")
#device = torch.device("cpu")

print(device)

cuda:0


In [2]:
#Common image transformations:
transform = transforms.Compose(
    #convert PIL image to tensor array:
    [transforms.ToTensor(),
    #each of the three pixel maps is centered to have mean 0.5 and std 0.5:
     transforms.Normalize((0.5,), (0.5,))])

#Download MNIST data:
#root: where data to be stored, if not exists already
#train=True: data is used for training (otherwise test set is created)
#download=True: downloads data, if not exists in root already
#transform: applies transformations to image
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

#Dataloader makes dataset an iterable object
#batch_size: how many samples per batch to load
#shuffle: have data reshuffled at every epoch
#num_workers: how many subprocesses to use for data loading (0 means data is loaded into main process)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

#Load test data set:
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
#Dataloader for test set:
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=True, num_workers=2)

#Set classes:
#class_label[0]=0, ..., class_label[9]=9:
classes = (0,1,2,3,4,5,6,7,8,9)

Exploring training set and test set:

In [3]:
print("Training set:", trainloader.dataset.data.shape)
print("Classes:", trainset.classes)
print("Test set:", testloader.dataset.data.shape)

for i in np.unique(np.asarray(trainloader.dataset.targets)):
    print("Class {0} has {1} training images and {2} test images".format(i, (trainloader.dataset.targets == i).sum(), (testloader.dataset.targets == i).sum()))

Training set: torch.Size([60000, 28, 28])
Classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
Test set: torch.Size([10000, 28, 28])
Class 0 has 5923 training images and 980 test images
Class 1 has 6742 training images and 1135 test images
Class 2 has 5958 training images and 1032 test images
Class 3 has 6131 training images and 1010 test images
Class 4 has 5842 training images and 982 test images
Class 5 has 5421 training images and 892 test images
Class 6 has 5918 training images and 958 test images
Class 7 has 6265 training images and 1028 test images
Class 8 has 5851 training images and 974 test images
Class 9 has 5949 training images and 1009 test images


## Sample example image batch

6     6     6     9


In [4]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


#Get some random training images:
#Make trainloader iteraterable (over batches, so 4 images are fetched): 
dataiter = iter(trainloader)
#returns random next element:
images, labels = dataiter.next()

#Show images:
imshow(torchvision.utils.make_grid(images))
#Print labels:
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

#images, labels = dataiter.next

## CNN architecture

In [5]:
#Define CNN architecture:

import torch.nn as nn
import torch.nn.functional as F

#Define Net as child class of nn.Module:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.dropout2 = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        #reshape input data x to be 1*784 vector:
        #x = x.view(-1, 28 * 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.dropout1(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (dropout2): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

## Loss function

In [6]:
#Define Loss function:

import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)

## Training

In [7]:
%%time

#Training:
epochs = 2

#loop over the dataset "epochs" times:
for epoch in range(epochs):

    running_loss = 0.0
    #enumerate(trainloader,0): make trainloader iterable and start the index at 0:
    for i, data in enumerate(trainloader, 0):
        #get the inputs; data is a list of [inputs, labels]:
        inputs, labels = data[0].to(device), data[1].to(device)

        #inputs = inputs.view(-1, 28 * 28)
        
        #zero the parameter gradients (just to be save):
        optimizer.zero_grad()

        #forward:
        outputs = net(inputs)

        #loss += F.nll_loss(outputs, labels, reduction='sum').item()

        #loss:
        loss = criterion(outputs, labels)

        #backward:
        loss.backward()
        #optimize:
        optimizer.step()

        #print statistics:
        running_loss += loss.item()

        #loss /= len(trainloader.dataset)

        #print every 2000 mini-batches:
        if i % 2000 == 1999:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch+1, i * len(data), len(trainloader.dataset), 100. * i / len(trainloader), running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Train Epoch: 1 [3998/60000 (13%)]	Loss: 0.476833
Train Epoch: 1 [7998/60000 (27%)]	Loss: 0.171895
Train Epoch: 1 [11998/60000 (40%)]	Loss: 0.139742
Train Epoch: 1 [15998/60000 (53%)]	Loss: 0.115490
Train Epoch: 1 [19998/60000 (67%)]	Loss: 0.101016
Train Epoch: 1 [23998/60000 (80%)]	Loss: 0.094308
Train Epoch: 1 [27998/60000 (93%)]	Loss: 0.091499
Train Epoch: 2 [3998/60000 (13%)]	Loss: 0.069660
Train Epoch: 2 [7998/60000 (27%)]	Loss: 0.069326
Train Epoch: 2 [11998/60000 (40%)]	Loss: 0.064501
Train Epoch: 2 [15998/60000 (53%)]	Loss: 0.067638
Train Epoch: 2 [19998/60000 (67%)]	Loss: 0.062832
Train Epoch: 2 [23998/60000 (80%)]	Loss: 0.063038
Train Epoch: 2 [27998/60000 (93%)]	Loss: 0.052780
Finished Training
CPU times: user 2min 11s, sys: 7.43 s, total: 2min 18s
Wall time: 2min 24s


In [8]:
#Save trained model parameters:

PATH = './mnist_cnn_net.pth'
torch.save(net.state_dict(), PATH)

## Calculate accuracy on test set:

In [9]:
#Count correctly classified images of trained data set:

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        
        images, labels = data[0].to(device), data[1].to(device)

        #do predicts on test set:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        #add the number of predicted images to "total", so we can keep track of how many images we have classified so far:
        total += labels.size(0)
        #add how many of the predicted labels are the same as the ground-truth labels:
        #(predicted == labels): predicted is tensor with integers, as well as labels; check if all integers are the same
        #(predicted == labels).sum(): sum up of how many were the same
        #(predicted == labels).sum().item(): give that number as a single value;
        #add it to the number of correctly classified instances:
        correct += (predicted == labels).sum().item()

#Print percentage of correctly classified instances:
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test images: 98 %


In [10]:
#Classes that performed well vs. classes that did not perform well:

#Initialize lists pf ten values of 0.0:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)

        #do predictions on test set:
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        #store correctly predicted classes (where prediction matches label) in "c":
        c = (predicted == labels)

        for i in range(4):
            label = labels[i]
            #count up how a certain class was predicted (true of false):
            class_correct[label] += c[i].item()
            #count up how often this class was predicted:
            class_total[label] += 1

#Print percentage of correctly classifications of each class:
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     0 : 97 %
Accuracy of     1 : 99 %
Accuracy of     2 : 97 %
Accuracy of     3 : 98 %
Accuracy of     4 : 98 %
Accuracy of     5 : 96 %
Accuracy of     6 : 97 %
Accuracy of     7 : 99 %
Accuracy of     8 : 98 %
Accuracy of     9 : 96 %


## Run predictions on test samples

Run all cells above up until the Loss function.

Run these 4 cells below to sample 4 random images of test set (with ground-truth labels) and run the CNN predictions on them:

In [59]:
#Load back model:
PATH = './mnist_cnn_net.pth'

net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

GroundTruth:      3     5     1     7


In [63]:
#Fetch random batch image of test set:

dataiter = iter(testloader)
images, labels = dataiter.next()

#Show images:
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [64]:
#Run predictions:

#this returns one-hot-encoding:
outputs = net(images)

In [65]:
#max(outputs,1) will give the indices of the max values of the one-hot-encoding (our predictions), and also a gradient object that we ignore by storing in "_":
_ , predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

Predicted:      3     5     1     7
